In [2]:
import os
import sys
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
from mlflow.models.signature import infer_signature
import mlflow.sklearn

from ipywidgets import interact

import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


In [3]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual,pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [5]:
if __name__=="__main__":
    csv_url=("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-red.csv")

    try:
        data = pd.read_csv(csv_url, sep=";")
    except Exception as e :
        logger.exception("Unable to download the data")

    train, test = train_test_split(data)

    train_x = train.drop(["quality"], axis = 1)
    test_x = test.drop(["quality"], axis = 1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    alpha =  0.5
    l1_ratio =  0.5

    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2)  = eval_metrics(test_y, predicted_qualities)
        print("Elasticnet model (alpha={:f}, l1_ratio={:f})".format(alpha, l1_ratio))
        print(" RMSE: %s" % rmse)
        print(" MAE: %s" % mae)
        print(" R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        remote_server_uri = "http://ec2-3-115-116-130.ap-northeast-1.compute.amazonaws.com:5000/"
        mlflow.set_tracking_uri(remote_server_uri)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        if tracking_url_type_store!="file":
            mlflow.sklearn.log_model(
                lr,"model", registered_model_name="ElsaticnetWineModel"
            )
        else:
            mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.500000, l1_ratio=0.500000)
 RMSE: 0.7292807047037724
 MAE: 0.5819219296653622
 R2: 0.13769146568322188


2025/08/06 14:42:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/06 14:42:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElsaticnetWineModel'.
2025/08/06 14:42:30 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElsaticnetWineModel, version 1


🏃 View run dapper-sponge-930 at: http://ec2-3-115-116-130.ap-northeast-1.compute.amazonaws.com:5000/#/experiments/0/runs/48e90f983ff340dab500698d88be6ae4
🧪 View experiment at: http://ec2-3-115-116-130.ap-northeast-1.compute.amazonaws.com:5000/#/experiments/0


Created version '1' of model 'ElsaticnetWineModel'.
